# Step 1: Import Libraries and Load the CLIP Model

In [9]:
import torch
import os
import clip
from PIL import Image, UnidentifiedImageError
import requests
from io import BytesIO
import mysql.connector
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv

In [ ]:
# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Step 2: Connect to TiDB

In [ ]:
def reconnect():
    try:
        connection = mysql.connector.connect(
            host=os.getenv('MYSQL_HOST'),
            user=os.getenv('MYSQL_USER'),
            password=os.getenv('MYSQL_PASSWORD'),
            database=os.getenv('MYSQL_DATABASE'),
            port=int(os.getenv('MYSQL_PORT'))
        )
        if connection.is_connected():
            print("Successfully connected to TiDB")
            return connection
    except mysql.connector.Error as e:
        print(f"Error: {e}")
        return None

# Establish the initial connection
connection = reconnect()
cursor = connection.cursor(dictionary=True)